<a href="https://colab.research.google.com/github/johnmatsson/MachineLearningNotebooks/blob/master/MNISTdigitsCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision import datasets

# Define the model (same as in the original code)
class convmodel1(nn.Module):
    def __init__(self, num_classes, hidden_size=256):
        super(convmodel1, self).__init__()

        self.num_classes = num_classes
        self.hidden_size = hidden_size

        self.conv1 = nn.Conv2d(in_channels=1, out_channels=64, kernel_size=(3, 3), stride=1)  # MNIST images are grayscale (1 channel)
        self.relu1 = nn.ReLU()
        self.avgpool = nn.AvgPool2d(kernel_size=(3, 3), stride=1, padding=0,  ceil_mode=False)
        self.conv2 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(5, 5), stride=1, padding=0, dilation=1, bias=False)
        self.batchnorm2d = nn.BatchNorm2d(num_features=128)
        self.conv3 = nn.Conv2d(in_channels=128, out_channels=64, kernel_size=(5, 5), stride=1, padding=0, dilation=1)
        self.relu2 = nn.ReLU()
        self.dropout = nn.Dropout(p=0.2)
        self.flatten = nn.Flatten()

        # Calculate the correct input size for fc1
        self.fc1 = nn.Linear(in_features=16384, out_features=512, bias=True)  # Updated Line!

        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(in_features=512, out_features=10)  # Output 10 classes for MNIST digits

    def forward(self, x):
        o1 = self.conv1(x)
        o2 = self.relu1(o1)
        o3 = self.avgpool(o2)
        o4 = self.conv2(o3)
        o5 = self.batchnorm2d(o4)
        o6 = self.conv3(o5)
        o7 = self.relu2(o6)
        o8 = self.dropout(o7)
        o9 = self.flatten(o8)
        o10 = self.fc1(o9)
        o11 = self.relu3(o10)
        o12 = self.fc2(o11)
        o13 = torch.log_softmax(o12, dim=1)  # Log-softmax for multi-class classification
        return o13

# Load and preprocess MNIST dataset
transform = transforms.Compose([
    transforms.ToTensor(),  # Convert images to PyTorch tensors
    transforms.Normalize((0.5,), (0.5,))  # Normalize with mean=0.5, std=0.5 for grayscale images
])

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Instantiate the model, loss function, and optimizer
model = convmodel1(num_classes=10)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

criterion = nn.NLLLoss()  # Negative log likelihood loss (commonly used with
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 5
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)

        # Calculate the loss
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        # Calculate accuracy
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    # Print loss and accuracy for this epoch
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, Accuracy: {100 * correct/total:.2f}%")

# Testing the model
model.eval()  # Set the model to evaluation mode
correct = 0
total = 0

with torch.no_grad():  # No need to track gradients during evaluation
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        # Forward pass
        outputs = model(inputs)

        # Get predicted labels
        _, predicted = torch.max(outputs, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# Print the test accuracy
print(f"Test Accuracy: {100 * correct/total:.2f}%")


Epoch [1/5], Loss: 0.1717, Accuracy: 95.39%


KeyboardInterrupt: 